In [39]:
# Extractive QA : 책에서 답을 찾는다
# 각 단어가 답의 시작일 확률 계산 start_logits
# 각 단어가 답의 끝일 확률 계산   end_logits
# 가장 확률이 높은 구간 선택      argmax

# Extractive QA 시각화
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import matplotlib.pyplot as plt
import numpy as np

In [40]:
# 모델 로드
MODEL_NAME = 'distilbert-base-cased-distilled-squad'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

In [41]:
question = "What is the capital of France?"
# question = "What is the population of France?"
context = "Paris is the capital and largest city of France. The city has a population of 2.1 million."

In [42]:
# tokenization
inputs = tokenizer(question, context, return_tensors='pt')

In [43]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] What is the capital of Korea? [SEP] Paris is the capital and largest city of France. The city has a population of 2. 1 million. [SEP]'

In [44]:
# 예측
with torch.no_grad():
    outputs = model(**inputs)
outputs
# 토큰 리스트
tokens = tokenizer.convert_ids_to_tokens( inputs['input_ids'][0])
tokens[:10]

['[CLS]', 'What', 'is', 'the', 'capital', 'of', 'Korea', '?', '[SEP]', 'Paris']

In [45]:
# logits를 확률로 변환
start_probs = torch.softmax(outputs.start_logits, dim=-1)[0].numpy()
end_probs = torch.softmax(outputs.end_logits, dim=-1)[0].numpy()
# 답변 추출(인덱스 추출) -> 생성하는게 아니라 답변과 가장 유사한 문맥을 찾아주는 것
answer_start = np.argmax(start_probs)
answer_end = np.argmax(end_probs) + 1
answer_tokens = tokens[answer_start:answer_end]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print(f'모델의 답변 : {answer}')
print(f'시작 위치 : {answer_start}, 토큰 : {tokens[answer_start]}')
print(f'종료 위치 : {answer_end}, 토큰 : {tokens[answer_end - 1]}')
print(f'신뢰도 : {start_probs[answer_start] * end_probs[answer_end - 1]:.4f}')

모델의 답변 : Paris
시작 위치 : 9, 토큰 : Paris
종료 위치 : 10, 토큰 : Paris
신뢰도 : 0.9666
